In [ ]:
# !Convert*dataset.py*w*sh*

In [ ]:
import time
import random
import math
import csv

import numpy
import scipy.stats

In [ ]:
default_sigma = 1.

In [ ]:
class OTProblem(object):
    def __init__(self, mu=None, nu=None, c=None):
        self.mu = mu
        self.nu = nu
        self.c = c
        self.ubox = None
        self.s = None
        self.sx = None
        self.objx = None
    
    def set_sx(self):
        self.sx = self.s
        self.objx = (self.s * self.c).sum()
    
    def set_objx(self):
        self.objx = (self.s * self.c).sum()
    
    def clean(self):
        self.s = None
    
    def plot_hotline(self, fh, colorbar=True, *args, **kwargs):
        mpbl = fh.ax.imshow(self.s, *args, **kwargs)
        if colorbar:
            fh.fig.colorbar(mpbl)
        return mpbl

In [ ]:
class OTProblem1d(OTProblem):
    def __init__(self, *args, **kwargs):
        super(OTProblem1d, self).__init__(*args, **kwargs)
    
    def plot_mu_scatter(self, fh, *args, **kwargs):
        fh.ax.scatter(self.mup, self.mu, *args, **kwargs)
    
    def plot_nu_scatter(self, fh, *args, **kwargs):
        fh.ax.scatter(self.nup, self.nu, *args, **kwargs)
    
    def plot_scatter(
        self, fh,
        cutoff=0.2,
        aspect="auto", colorbar=True,
        *args, **kwargs
    ):
        m, n = self.c.shape
        
        mean = self.s.mean() * numpy.sqrt(m * n)
        mask = self.s > cutoff * mean
        
        ind = numpy.indices((m, n))
        
        dot_x, dot_y = self.mup[ind[0]][mask], self.nup[ind[1]][mask]
        dot_c = self.s[mask]
        
        mpbl = fh.ax.scatter(dot_x, dot_y, c=dot_c, *args, **kwargs)
        if colorbar:
            fh.fig.colorbar(mpbl)
        
        fh.ax.set_aspect(aspect)
        return mpbl
    
    def plot_link(
        self, fh,
        off_bx=0., off_by=0., off_ex=0., off_ey=1.,
        scatter=True, 
        cutoff=0.2, scale=30.,
        aspect="auto", colorbar=True,
        sca_pos=(), sca_kw={},
        *args, **kwargs
    ):
        m, n = self.c.shape
        
        if scatter:
            mus = self.mu / self.mu.mean() * scale
            nus = self.nu / self.nu.mean() * scale
            fh.ax.scatter(self.mup + off_bx, numpy.zeros_like(self.mup) + off_by, s=mus, *sca_pos, **sca_kw)
            fh.ax.scatter(self.nup + off_ex, numpy.zeros_like(self.nup) + off_ey, s=nus, *sca_pos, **sca_kw)
        
        mean = self.s.mean() * numpy.sqrt(m * n)
        mask = self.s > cutoff * mean
        
        ind = numpy.indices((m, n))

        arr_bx = self.mup[ind[0]][mask] + off_bx
        arr_by = numpy.zeros_like(arr_bx) + off_by

        arr_ex = self.nup[ind[1]][mask] + off_ex
        arr_ey = numpy.zeros_like(arr_ex) + off_ey

        arr_dx = arr_ex - arr_bx
        arr_dy = arr_ey - arr_by

        arr_c = self.s[mask]

        mpbl = fh.ax.quiver(arr_bx, arr_by, arr_dx, arr_dy, arr_c, angles="xy", scale_units="xy", scale=1., *args, **kwargs)
        if colorbar:
            fh.fig.colorbar(mpbl)
        
        fh.ax.set_aspect(aspect)
        return mpbl

In [ ]:
def samp_1d_grid(start, stop):
    def samp_1d_grid_gen(num):
        p = numpy.linspace(start, stop, num)
        return p
    return samp_1d_grid_gen

In [ ]:
def samp_1d_norm(loc, scale):
    def samp_1d_norm_gen(num):
        p = numpy.random.normal(loc, scale, num)
        p.sort()
        return p
    return samp_1d_norm_gen

In [ ]:
def val_const(sigma=default_sigma):
    def val_const_gen(p):
        num = p.shape[0]
        v = numpy.ones(num) * sigma / num
        return v
    return val_const_gen

In [ ]:
def val_unif(low, high, sigma=default_sigma):
    def val_unif_gen(p):
        num = p.shape[0]
        v = numpy.random.uniform(low, high, num)
        v = v * sigma / numpy.sum(v)
        return v
    return val_unif_gen

In [ ]:
def val_1d_norm_pdf(loc, scale, sigma=default_sigma):
    norm = scipy.stats.norm(loc=loc, scale=scale)
    def val_1d_norm_pdf_gen(p):
        v = norm.pdf(p)
        v = v * sigma / numpy.sum(v)
        return v
    return val_1d_norm_pdf_gen

In [ ]:
def val_1d_norm_pdf_2(loc1, scale1, loc2, scale2, sigma=default_sigma):
    norm1 = scipy.stats.norm(loc=loc1, scale=scale1)
    norm2 = scipy.stats.norm(loc=loc2, scale=scale2)
    def val_1d_norm_pdf_2_gen(p):
        v1 = norm1.pdf(p)
        v2 = norm2.pdf(p)
        v = v1 + v2
        v = v * sigma / numpy.sum(v)
        return v
    return val_1d_norm_pdf_2_gen

In [ ]:
def dist_1d_euc_2(mup, nup):
    m, n = mup.size, nup.size
    ind = numpy.indices((m, n))
    c = (mup[ind[0]] - nup[ind[1]])**2
    return c

In [ ]:
def ot_1d_general(m, n, mu_gen, nu_gen, mup_gen, nup_gen, dist, seed):
    random.seed(seed)
    numpy.random.seed(seed)
    p = OTProblem1d()
    p.mup = mup_gen(m)
    p.nup = nup_gen(n)
    p.mu = mu_gen(p.mup)
    p.nu = nu_gen(p.nup)
    p.c = dist(p.mup, p.nup)
    return p

In [ ]:
class OTProblem2d(OTProblem):
    def __init__(self, *args, **kwargs):
        super(OTProblem2d, self).__init__(*args, **kwargs)
    
    def plot_mu_scatter(self, fh, *args, **kwargs):
        fh.ax.scatter(self.mup[:, 0], self.mup[:, 1], self.mu, *args, **kwargs)
    
    def plot_nu_scatter(self, fh, *args, **kwargs):
        fh.ax.scatter(self.nup[:, 0], self.nup[:, 1], self.nu, *args, **kwargs)
    
    def plot_mu_scatter_plain(self, fh, scale=30., *args, **kwargs):
        s = self.mu / self.mu.mean() * scale
        fh.ax.scatter(self.mup[:, 0], self.mup[:, 1], s=s, *args, **kwargs)
    
    def plot_nu_scatter_plain(self, fh, scale=30., *args, **kwargs):
        s = self.nu / self.nu.mean() * scale
        fh.ax.scatter(self.nup[:, 0], self.nup[:, 1], s=s, *args, **kwargs)
    
    def plot_link(
        self, fh,
        off_bx=0., off_by=0., off_ex=0., off_ey=0.,
        scatter=True,
        cutoff=0.2, scale=30.,
        aspect="auto", colorbar=True,
        sca_pos=(), sca_kw={},
        *args, **kwargs
    ):
        m, n = self.c.shape
        
        if scatter:
            mus = self.mu / self.mu.mean() * scale
            nus = self.nu / self.nu.mean() * scale
            fh.ax.scatter(self.mup[:, 0] + off_bx, self.mup[:, 1] + off_by, s=mus, *sca_pos, **sca_kw)
            fh.ax.scatter(self.nup[:, 0] + off_ex, self.nup[:, 1] + off_ey, s=nus, *sca_pos, **sca_kw)
        
        mean = self.s.mean() * numpy.sqrt(m * n)
        mask = self.s> cutoff * mean

        ind = numpy.indices((m, n))

        arr_b = self.mup[ind[0]][mask]
        arr_bx, arr_by = arr_b[:, 0] + off_bx, arr_b[:, 1] + off_by

        arr_e = self.nup[ind[1]][mask]
        arr_ex, arr_ey = arr_e[:, 0] + off_ex, arr_e[:, 1] + off_ey

        arr_dx = arr_ex - arr_bx
        arr_dy = arr_ey - arr_by

        arr_c = self.s[mask]

        mpbl = fh.ax.quiver(arr_bx, arr_by, arr_dx, arr_dy, arr_c, angles="xy", scale_units="xy", scale=1., *args, **kwargs)
        if colorbar:
            fh.fig.colorbar(mpbl)
        
        fh.ax.set_aspect(aspect)
        return mpbl

In [ ]:
def samp_norm(mean, cov):
    def samp_norm_gen(num):
        p = numpy.random.multivariate_normal(mean, cov, num)
        return p
    return samp_norm_gen

In [ ]:
def samp_2d_unif(size):
    def samp_2d_unif(num):
        p = size * (numpy.random.rand(num*2).reshape((num, 2)))
        return p
    return samp_2d_unif

In [ ]:
def samp_2d_grid(start_x, end_x, start_y, end_y):
    def samp_2d_grid_gen(num):
        num_t = num[0] * num[1]
        x = numpy.linspace(start_x, end_x, num[0])
        y = numpy.linspace(start_y, end_y, num[1])
        xp, yp = numpy.meshgrid(x, y)
        p = numpy.concatenate((xp.reshape((num_t, 1)), yp.reshape((num_t, 1))), axis=1)
        return p
    return samp_2d_grid_gen

In [ ]:
def samp_2d_mid(start_x, end_x, start_y, end_y):
    def samp_2d_grid_gen(num):
        num_t = num[0] * num[1]
        step_x = (end_x - start_x) / num[0]
        x = numpy.linspace(start_x + step_x / 2., end_x - step_x / 2., num[0])
        step_y = (end_y - start_y) / num[1]
        y = numpy.linspace(start_y + step_y / 2., end_y - step_y / 2., num[1])
        xp, yp = numpy.meshgrid(x, y)
        p = numpy.concatenate((xp.reshape((num_t, 1)), yp.reshape((num_t, 1))), axis=1)
        return p
    return samp_2d_grid_gen

In [ ]:
def samp_2d_ellipse(cen_x, r_x, cen_y, r_y, noi):
    def samp_2d_ellipse_gen(num):
        r = numpy.random.uniform(0, 2.*math.pi, num)
        dx = numpy.cos(r) + noi / math.sqrt(2.) * numpy.random.randn(num)
        dy = numpy.sin(r) + noi / math.sqrt(2.) * numpy.random.randn(num)
        x = r_x * dx + cen_x
        y = r_y * dy + cen_y
        p = numpy.concatenate((x.reshape(num, 1), y.reshape(num, 1)), axis=1)
        return p
    return samp_2d_ellipse_gen

In [ ]:
def samp_2d_Caffarelli(cen_x, cen_y, r, d):
    def samp_2d_Caffarelli_gen(num):
        ox = numpy.random.uniform(-r, r, num)
        oy = numpy.random.uniform(-r, r, num)
        mask = ox**2 + oy**2 < r**2
        fx, fy = ox[mask], oy[mask]
        fx[fx < 0.] -= d / 2.
        fx[fx >= 0.] += d / 2.
        nnum = fx.size
        x = fx + cen_x
        y = fy + cen_y
        p = numpy.concatenate((x.reshape(nnum, 1), y.reshape(nnum, 1)), axis=1)
        return p
    return samp_2d_Caffarelli_gen

In [ ]:
def val_norm_pdf(mean, cov, sigma=default_sigma):
    norm = scipy.stats.multivariate_normal(mean=mean, cov=cov)
    def val_norm_pdf_gen(p):
        v = norm.pdf(p)
        v = v * sigma / numpy.sum(v)
        return v
    return val_norm_pdf_gen

In [ ]:
def dist_2d_euc_2(mup, nup):
    m, n = mup.shape[0], nup.shape[0]
    ind = numpy.indices((m, n))
    c = ((mup[ind[0]] - nup[ind[1]])**2).sum(axis=2)
    return c

In [ ]:
def ot_2d_general(m, n, mu_gen, nu_gen, mup_gen, nup_gen, dist, seed):
    random.seed(seed)
    numpy.random.seed(seed)
    p = OTProblem2d()
    p.mup = mup_gen(m)
    p.nup = nup_gen(n)
    p.mu = mu_gen(p.mup)
    p.nu = nu_gen(p.nup)
    p.c = dist(p.mup, p.nup)
    return p

In [ ]:
def ot_2d_random(m, n, seed):
    prob = ot_2d_general(
        m=m, n=n,
        mup_gen=samp_2d_unif(1.),
        nup_gen=samp_2d_unif(1.),
        mu_gen=val_unif(0., 1.),
        nu_gen=val_unif(0., 1.),
        dist=dist_2d_euc_2,
        seed=seed,
    )
    return prob

In [ ]:
def ot_2d_ellipse(m, n, seed):
    prob = ot_2d_general(
        m=m, n=n,
        mup_gen=samp_2d_ellipse(0., 2., 0., 0.5, 0.1),
        nup_gen=samp_2d_ellipse(0., 0.5, 0., 2., 0.1),
        mu_gen=val_const(),
        nu_gen=val_const(),
        dist=dist_2d_euc_2,
        seed=seed,
    )
    return prob

In [ ]:
def ot_2d_Caffarelli(m, n, seed):
    prob = ot_2d_general(
        m=m, n=n,
        mup_gen=samp_2d_Caffarelli(0., 0., 1., 0.),
        nup_gen=samp_2d_Caffarelli(0., 0., 1., 2.),
        mu_gen=val_const(),
        nu_gen=val_const(),
        dist=dist_2d_euc_2,
        seed=seed,
    )
    return prob

In [ ]:
class OTProblemImage(OTProblem2d):
    def __init__(self, *args, **kwargs):
        super(OTProblemImage, self).__init__(*args, **kwargs)
    
    def plot_mu_image(
        self, fh,
        origin="lower",
        colorbar=False,
        *args, **kwargs
    ):
        mwn = [self.mw[0], self.mw[1], self.mw[2], self.mw[3]]
        mpbl = fh.ax.imshow(self.mu.reshape(self.m[1], self.m[0]), extent=mwn, origin=origin, *args, **kwargs)
        if colorbar:
            fh.fig.colorbar(mpbl)
    
    def plot_nu_image(
        self, fh,
        origin="lower",
        colorbar=False,
        *args, **kwargs
    ):
        nwn = [self.nw[0], self.nw[1], self.nw[2], self.nw[3]]
        mpbl = fh.ax.imshow(self.nu.reshape(self.n[1], self.n[0]), extent=nwn, origin=origin, *args, **kwargs)
        if colorbar:
            fh.fig.colorbar(mpbl)
    
    def plot_link(
        self, fh,
        off_bx=0., off_by=0., off_ex=0., off_ey=0.,
        image_m=False, image_n=False,
        origin="lower",
        cutoff=0.2, scale=30.,
        aspect="auto", colorbar=False,
        im_pos=(), im_kw={},
        *args, **kwargs
    ):
        m, n = self.c.shape
        
        if image_m:
            mwn = [self.mw[0] + off_bx, self.mw[1] + off_bx, self.mw[2] + off_by, self.mw[3] + off_by]
            fh.ax.imshow(self.mu.reshape(self.m), extent=mwn, origin=origin, *im_pos, **im_kw)
        
        if image_n:
            nwn = [self.nw[0] + off_ex, self.nw[1] + off_ex, self.nw[2] + off_ey, self.nw[3] + off_ey]
            fh.ax.imshow(self.nu.reshape(self.n), extent=nwn, origin=origin, *im_pos, **im_kw)
        
        mean = self.s.mean() * numpy.sqrt(m * n)
        mask = self.s> cutoff * mean

        ind = numpy.indices((m, n))

        arr_b = self.mup[ind[0]][mask]
        arr_bx, arr_by = arr_b[:, 0] + off_bx, arr_b[:, 1] + off_by

        arr_e = self.nup[ind[1]][mask]
        arr_ex, arr_ey = arr_e[:, 0] + off_ex, arr_e[:, 1] + off_ey

        arr_dx = arr_ex - arr_bx
        arr_dy = arr_ey - arr_by

        arr_c = self.s[mask]

        mpbl = fh.ax.quiver(arr_bx, arr_by, arr_dx, arr_dy, arr_c, angles="xy", scale_units="xy", scale=1., *args, **kwargs)
        if colorbar:
            fh.fig.colorbar(mpbl)
        
        fh.ax.set_aspect(aspect)
        return mpbl

In [ ]:
def ot_im_general(m, n, mw, nw, mu_gen, nu_gen, mup_gen, nup_gen, dist, seed):
    random.seed(seed)
    numpy.random.seed(seed)
    p = OTProblemImage()
    p.m = m
    p.n = n
    p.mw = mw
    p.nw = nw
    p.mup = mup_gen(m)
    p.nup = nup_gen(n)
    p.mu = mu_gen(p.mup)
    p.nu = nu_gen(p.nup)
    p.c = dist(p.mup, p.nup)
    return p

In [ ]:
folder_name = [
    "WhiteNoise",
    "GRFrough",
    "GRFmoderate",
    "GRFsmooth",
    "LogGRF",
    "LogitGRF",
    "CauchyDensity",
    "Shapes",
    "ClassicImages",
    "MicroscopyImages"
]

In [ ]:
def val_dotmark(cls, log=None, sigma=default_sigma):
    def val_dotmark_gen(p):
        s = int(math.sqrt(p.shape[0]))
        i = random.randint(1, 10)
        if log is not None:
            log("Image {0} is chosen".format(i))
        fn = "Data/{0}/data{1}_1{2:03}.csv".format(folder_name[cls], s, i)
        with open(fn) as f:
            rd = csv.reader(f)
            l = []
            for r in rd:
                l.append([int(v) for v in r])
        a = numpy.array(l)
        a = a * sigma / a.sum()
        return a.reshape(s**2)
    return val_dotmark_gen

In [ ]:
def ot_im_dotmark(cls, size, seed, log=None):
    prob = ot_im_general(
        m=[size, size], n=[size, size],
        mw=[0., 1., 0., 1.], nw=[0., 1., 0., 1.],
        mup_gen=samp_2d_mid(0., 1., 0., 1.),
        nup_gen=samp_2d_mid(0., 1., 0., 1.),
        mu_gen=val_dotmark(cls, log=log),
        nu_gen=val_dotmark(cls, log=log),
        dist=dist_2d_euc_2,
        seed=seed,
    )
    return prob

In [ ]:
# !ConvertEnd*

In [ ]:
# !Convert*dataset_test.py*w*sehx*

In [ ]:
# !Switch*
from handler import FigureHandler
# !SwitchCase*
# import font
# from handler import FigureHandler
# from dataset import *
# !SwitchEnd*

In [ ]:
# !Switch*
fh = FigureHandler(redir=True)
# !SwitchCase*
# fh = FigureHandler(sav=True, disp=False, ext=".pgf", redir=True)
# !SwitchEnd*

In [ ]:
import mosek

def mosek_set_model(p, task):
    m, n = p.c.shape
    
    inf = 0.
    
    task.appendvars(m*n)
    task.appendcons(m+n)
    
    task.putvarboundlist(
        range(m*n),
        [mosek.boundkey.lo]*(m*n),
        [0.]*(m*n),
        [inf]*(m*n)
    )
    
    for i in range(m):
        task.putarow(
            i,
            range(i*n, (i+1)*n),
            [1.]*n
        )
    task.putconboundlist(
        range(0, m),
        [mosek.boundkey.fx]*m,
        p.mu,
        p.mu
    )
    
    for i in range(n):
        task.putarow(
            i+m,
            range(i, i+m*n, n),
            [1.]*m
        )
    task.putconboundlist(
        range(m, m+n),
        [mosek.boundkey.fx]*n,
        p.nu,
        p.nu
    )
    
    task.putclist(range(m*n), p.c.reshape(m*n))
    
    task.putobjsense(mosek.objsense.minimize)

def solve_mosek_primal_simplex(
    p,
    log=None, stat=False,
    *args, **kwargs
):
    m, n = p.c.shape
    
    if stat:
        start_time = time.time()
    
    with mosek.Env() as env:
        env.set_Stream(mosek.streamtype.log, log)
        
        with env.Task() as task:
            task.set_Stream(mosek.streamtype.log, log)
            
            task.putintparam(mosek.iparam.optimizer, mosek.optimizertype.primal_simplex)
        
            mosek_set_model(p, task)
            
            if stat:
                end_time = time.time()
            
            task.optimize()
            
            xx = [0.] * (m*n)
            task.getxx(mosek.soltype.bas, xx)
            
            p.s = numpy.array(xx).reshape(m, n)
    
            if stat:
                setup_time = end_time - start_time
                s = {
                    "loss": task.getprimalobj(mosek.soltype.bas),
                    "vars": task.getintinf(mosek.iinfitem.opt_numvar),
                    "iters": task.getintinf(mosek.iinfitem.sim_primal_iter),
                    "setup": setup_time,
                    "solve": task.getdouinf(mosek.dinfitem.optimizer_time)
                }
                return p, s
            else:
                return p

In [ ]:
prob = OTProblem()
prob.s = numpy.fromfunction(lambda i, j: numpy.sin(i / 3) * j**2, (50, 50))

In [ ]:
fh.fast(prob.plot_hotline)

In [ ]:
prob = OTProblem1d()
prob.mup = numpy.linspace(0., 1., 50)
prob.nup = numpy.linspace(2., 3., 50)
prob.mu = numpy.linspace(2., 1., 50)
prob.nu = numpy.linspace(4., 3., 50)
prob.c = numpy.eye(50)
prob.s= numpy.eye(50)

In [ ]:
fh.fast(prob.plot_mu_scatter, alpha=0.5)

In [ ]:
fh.fast(prob.plot_scatter, alpha=0.5)

In [ ]:
prob = ot_1d_general(
    m=50, n=50,
    mup_gen=samp_1d_norm(-5., 1.),
    nup_gen=samp_1d_grid(-5., -4.),
    mu_gen=val_unif(0., 1.),
    nu_gen=val_unif(0., 1.),
    dist=dist_1d_euc_2,
    seed=1,
)

In [ ]:
fh.fast(prob.plot_mu_scatter)

In [ ]:
_, s = solve_mosek_primal_simplex(prob, stat=True)
fh.write(repr(s))

In [ ]:
fh.fast(prob.plot_hotline)

In [ ]:
fh.fast(prob.plot_scatter, alpha=0.5)

In [ ]:
fh.fast(prob.plot_link)

In [ ]:
prob = ot_1d_general(
    m=50, n=100,
    mup_gen=samp_1d_grid(-7., -3.),
    nup_gen=samp_1d_grid(-3., -1.),
    mu_gen=val_1d_norm_pdf(-5., 0.8),
    nu_gen=val_1d_norm_pdf(-2., 0.4),
    dist=dist_1d_euc_2,
    seed=1,
)

In [ ]:
fh.fast(prob.plot_mu_scatter)

In [ ]:
solve_mosek_primal_simplex(prob)

In [ ]:
fh.fast(prob.plot_link)

In [ ]:
fh.fast(prob.plot_hotline)

In [ ]:
prob = ot_2d_general(
    m=[5, 5], n=50,
    mup_gen=samp_2d_mid(0., 1., 0., 1.),
    nup_gen=samp_norm([2., 2.], [[1., 0.5], [0.5, 1.]]),
    mu_gen=val_unif(0., 1.),
    nu_gen=val_unif(0., 1.),
    dist=dist_2d_euc_2,
    seed=1,
)

In [ ]:
solve_mosek_primal_simplex(prob)

In [ ]:
fh.fast(prob.plot_hotline)

In [ ]:
fh.fast(prob.plot_mu_scatter, new_kw={"projection": "3d"})

In [ ]:
fh.fast(prob.plot_mu_scatter_plain)

In [ ]:
fh.fast(prob.plot_nu_scatter_plain)

In [ ]:
fh.fast(prob.plot_link, scale=30., cutoff=0.5)

In [ ]:
prob = ot_2d_general(
    m=100, n=100,
    mup_gen=samp_2d_ellipse(0., 2., 0., 0.5, 0.1),
    nup_gen=samp_2d_ellipse(0., 0.5, 0., 2., 0.1),
    mu_gen=val_const(),
    nu_gen=val_const(),
    dist=dist_2d_euc_2,
    seed=1,
)

In [ ]:
fh.fast(prob.plot_mu_scatter_plain)

In [ ]:
solve_mosek_primal_simplex(prob)

In [ ]:
fh.fast(prob.plot_link, scale=30., cutoff=0.5)

In [ ]:
prob = ot_2d_general(
    m=500, n=500,
    mup_gen=samp_2d_Caffarelli(0., 0., 1., 0.),
    nup_gen=samp_2d_Caffarelli(0., 0., 1., 2.),
    mu_gen=val_const(),
    nu_gen=val_const(),
    dist=dist_2d_euc_2,
    seed=1,
)

In [ ]:
fh.fast(prob.plot_nu_scatter_plain)

In [ ]:
solve_mosek_primal_simplex(prob)

In [ ]:
fh.fast(prob.plot_link, scale=30., cutoff=0.5)

In [ ]:
prob = ot_im_general(
    m=[10, 10], n=[10, 10],
    mw=[0., 1., 0., 1.], nw=[0., 1., 0., 1.],
    mup_gen=samp_2d_mid(0., 1., 0., 1.),
    nup_gen=samp_2d_mid(0., 1., 0., 1.),
    mu_gen=val_unif(0., 1.),
    nu_gen=val_unif(0., 1.),
    dist=dist_2d_euc_2,
    seed=1,
)

In [ ]:
solve_mosek_primal_simplex(prob)

In [ ]:
fh.fast(prob.plot_link, aspect="equal")

In [ ]:
fh.refresh()
fh.subplot(1, 3, 1)
prob.plot_mu_image(fh)
fh.subplot(1, 3, 2)
prob.plot_link(fh, aspect="equal")
fh.subplot(1, 3, 3)
prob.plot_nu_image(fh)
fh.show()
fh.close()

In [ ]:
prob = ot_im_general(
    m=[20, 10], n=[10, 30],
    mw=[0., 2., 0., 1.], nw=[0., 1., 0., 2.],
    mup_gen=samp_2d_mid(0., 2., 0., 1.),
    nup_gen=samp_2d_mid(0., 1., 0., 2.),
    mu_gen=val_norm_pdf([0.2, 0.8], [[0.2, 0.], [0., 0.2]]),
    nu_gen=val_norm_pdf([0.8, 0.2], [[0.2, 0.], [0., 0.2]]),
    dist=dist_2d_euc_2,
    seed=1,
)

In [ ]:
solve_mosek_primal_simplex(prob)

In [ ]:
fh.fast(prob.plot_mu_image)

In [ ]:
fh.refresh()
fh.subplot(1, 3, 1)
prob.plot_mu_image(fh)
fh.subplot(1, 3, 2)
prob.plot_mu_image(fh, alpha=0.3)
prob.plot_nu_image(fh, alpha=0.3)
prob.plot_link(fh, aspect="equal")
fh.subplot(1, 3, 3)
prob.plot_nu_image(fh)
fh.show()
fh.close()

In [ ]:
prob = ot_2d_random(500, 500, seed=1)

In [ ]:
fh.fast(prob.plot_mu_scatter_plain)

In [ ]:
solve_mosek_primal_simplex(prob)

In [ ]:
fh.fast(prob.plot_link)

In [ ]:
prob = ot_im_dotmark(7, 32, seed=1, log=fh.write)

In [ ]:
fh.fast(prob.plot_mu_image)
fh.fast(prob.plot_nu_image)

In [ ]:
solve_mosek_primal_simplex(prob)

In [ ]:
fh.fast(prob.plot_link, aspect="equal")

In [ ]:
# !ConvertEnd*